In [1]:
import pandas as pd
import json
import re
import os
from bs4 import BeautifulSoup
import numpy as np

## DataFusion runtimes

Total runtimes of binary join & shredded yannakakis, aggregated per query.

In [2]:
df = pd.read_csv('timings_revision.csv')
df["total_time"] = np.nan # total time = optimization time + execution time (in seconds)
df['execution_time'] = df['duration(µs)'] / 1_000_000
df.drop(columns=['variant','duration(µs)'], inplace=True)
df.replace("BinaryJoin","DF-Bin", inplace=True)
df.replace("Yannakakis","SYA", inplace=True)
df_agg = df.groupby(["query","method"]).aggregate("median")
df_agg.reset_index(inplace=True)
df_agg

,query,method,total_time,execution_time
0,1,DF-Bin,NaN,0.001469
1,1,SYA,NaN,0.001255
2,2,DF-Bin,NaN,0.142300
3,2,SYA,NaN,0.020380
4,3,DF-Bin,NaN,0.026375
...,...,...,...,...
281,144,SYA,NaN,0.089205
282,145,DF-Bin,NaN,0.018423
283,145,SYA,NaN,0.015764
284,146,DF-Bin,NaN,0.035033


## Datafusion detailed metrics

In [3]:
def replace_utf8_string(text):
    # Use regex to match and replace value: Utf8("SomeString") by value: Utf8(\"SomeString\")
    pattern = r'value:\s*Utf8\("(.*?)"\)'  # This matches 'value: Utf8("SomeString")'
    replacement = r'value: Utf8(\"\1\")'  # Adds escaped quotes: Utf8(\"SomeString\")
    
    result = re.sub(pattern, replacement, text)
    
    return result

metrics_file = "output_revision/metrics.txt"
metrics = []
with open(metrics_file) as f:
    for i, line in enumerate(f):
        try:
            line = replace_utf8_string(line)
            metrics.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Error in line {i}")

# Drop 2NSA plan metrics (we're now analyzing binary joins)
metrics = [metric for metric in metrics if metric["params"]["method"] == "BinaryJoin"]
print(len(metrics)) # should be nr_of_queries * 10 (10 repetitions)

1430


In [4]:
def get_metric(metric_name, metrics):
    for metric in metrics:
        if metric["name"] == metric_name:
            return metric

def collect_metrics(metrics):
    def update_timings(node):
        if node["operator"].startswith("AggregateExec"):
            timings["aggregate_time"] += get_metric("elapsed_compute", node["metrics"])["value"]
        elif node["operator"].startswith("FilterExec"):
            timings["filter_time"] += get_metric("elapsed_compute", node["metrics"])["value"]
        elif node["operator"].startswith("ProjectionExec"):
            timings["projection_time"] += get_metric("elapsed_compute", node["metrics"])["value"]
        # memoryexec does not contain timing metrics
        elif node["operator"].startswith("ParquetExec"):
            timings["parquet_time"] += get_metric("time_elapsed_processing", node["metrics"])["value"]
        elif node["operator"].startswith("CoalesceBatchesExec"):
            timings["coalesce_batches_time"] += get_metric("elapsed_compute", node["metrics"])["value"]


        for child in node["children"]:
            update_timings(child)

    result = {}

    result["method"] = metrics["params"]["method"]
    result["query"] = int(metrics["params"]["query"])

    timings = {
        "aggregate_time": 0,
        "filter_time": 0,
        "projection_time": 0,
        "parquet_time": 0,
        "coalesce_batches_time": 0
    }
    root = metrics["plan"]
    update_timings(root)
    # all timings are in nanoseconds, convert to seconds
    timings = {k: v / 1_000_000_000 for k, v in timings.items()}
    result.update(timings)
    return result

df_bin = pd.DataFrame([collect_metrics(m) for m in metrics])
df_bin["method"] = "DF-Bin"
df_bin = df_bin.groupby(["query","method"]).aggregate("median")
df_bin.reset_index(inplace=True)
df_bin = pd.merge(
    df_agg[df_agg["method"] == "DF-Bin"],
    df_bin,
    on=["query","method"]
)
# hashjoin time = total time - aggregate time - filter time - projection time - parquet time - coalesce_batches_time
df_bin["hashjoin_time"] = df_bin["execution_time"] - df_bin["aggregate_time"] - df_bin["filter_time"] - df_bin["projection_time"] - df_bin["parquet_time"] - df_bin["coalesce_batches_time"]
df_bin

,query,method,total_time,execution_time,aggregate_time,filter_time,projection_time,parquet_time,coalesce_batches_time,hashjoin_time
0,1,DF-Bin,NaN,0.001469,0.000004,0.000048,0.000000,0.000282,0.000005,0.001131
1,2,DF-Bin,NaN,0.142300,0.000577,0.000965,0.000007,0.001987,0.000046,0.138718
2,3,DF-Bin,NaN,0.026375,0.000089,0.001310,0.000003,0.003067,0.000083,0.021824
3,4,DF-Bin,NaN,0.024219,0.000021,0.000801,0.000001,0.004442,0.000056,0.018898
4,5,DF-Bin,NaN,0.089819,0.000503,0.001654,0.000001,0.001921,0.000074,0.085666
...,...,...,...,...,...,...,...,...,...,...
138,142,DF-Bin,NaN,5.012924,0.036363,0.003102,0.000014,0.008145,0.000201,4.965099
139,143,DF-Bin,NaN,3.927998,0.026589,0.003197,0.000014,0.007737,0.000231,3.890231
140,144,DF-Bin,NaN,0.335587,0.004142,0.000934,0.000003,0.004470,0.000055,0.325982
141,145,DF-Bin,NaN,0.018423,0.000068,0.002313,0.000007,0.004693,0.000128,0.011213


In [5]:
# Time (sec) spent on ParquetExec
print(df_bin["parquet_time"].describe())

# Percentage of time spent on ParquetExec
print((df_bin["parquet_time"] / df_bin["execution_time"] * 100).describe())

count    143.000000
mean       0.004124
std        0.001745
min        0.000282
25%        0.002673
50%        0.004019
75%        0.005204
max        0.008145
Name: parquet_time, dtype: float64
count    143.000000
mean      17.345195
std       16.670749
min        0.007614
25%        2.217288
50%       12.810350
75%       26.730100
max       59.372973
dtype: float64


In [6]:
df_bin.drop(columns=["aggregate_time","filter_time","projection_time","parquet_time","coalesce_batches_time"], inplace=True)
df_bin.to_csv('timings_agg_revision.csv', index=False)
df_bin

,query,method,total_time,execution_time,hashjoin_time
0,1,DF-Bin,NaN,0.001469,0.001131
1,2,DF-Bin,NaN,0.142300,0.138718
2,3,DF-Bin,NaN,0.026375,0.021824
3,4,DF-Bin,NaN,0.024219,0.018898
4,5,DF-Bin,NaN,0.089819,0.085666
...,...,...,...,...,...
138,142,DF-Bin,NaN,5.012924,4.965099
139,143,DF-Bin,NaN,3.927998,3.890231
140,144,DF-Bin,NaN,0.335587,0.325982
141,145,DF-Bin,NaN,0.018423,0.011213


In [7]:
metrics_file = "output_revision/metrics.txt"
metrics = []
with open(metrics_file) as f:
    for i, line in enumerate(f):
        try:
            line = replace_utf8_string(line)
            metrics.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Error in line {i}")

projection = []

def filter_time(filternode):
    for metric in filternode["metrics"]:
        if metric["name"] == "elapsed_compute":
            return metric["value"]

    # filternode was never executed
    # can be due to early stopping in case of a multisemijoin with >=2 children.  
    return 0

def projection_time(projectionnode):
    for metric in projectionnode["metrics"]:
        if metric["name"] == "elapsed_compute":
            return metric["value"]

    # projectionnode was never executed
    # can be due to early stopping in case of a multisemijoin with >=2 children.  
    return 0

def aggregate_time(aggregatenode):
    for metric in aggregatenode["metrics"]:
        if metric["name"] == "elapsed_compute":
            return metric["value"]
    raise ValueError("aggregate_time metric not found")

def parquet_time(parquetnode):
    for metric in parquetnode["metrics"]:
        if metric["name"] == "time_elapsed_processing":
            return metric["value"]
    
    # parquetexec was never executed
    # can be due to early stopping in case of a multisemijoin with >=2 children.  
    return 0

def coalesce_batches_time(coalescenode):
    for metric in coalescenode["metrics"]:
        if metric["name"] == "elapsed_compute":
            return metric["value"]
        
    # coalescebatchesexec was never executed
    # can be due to early stopping in case of a multisemijoin with >=2 children.
    return 0
        
        
def collect_timings(node, timings: dict):
    if node["operator"].startswith("FilterExec"):
        timings["filter_time"] += filter_time(node)
    elif node["operator"].startswith("ProjectionExec"):
        timings["projection_time"] += projection_time(node)
    elif node["operator"].startswith("Aggregate"):
        timings["aggregate_time"] += aggregate_time(node)
    elif node["operator"].startswith("ParquetExec"):
        timings["parquet_time"] += parquet_time(node)
    elif node["operator"].startswith("CoalesceBatchesExec"):
        timings["coalesce_batches_time"] += coalesce_batches_time(node)

    for child in node["children"]:
        collect_timings(child, timings)


for entry in metrics:
    method = entry["params"]["method"]
    if method=="BinaryJoin": # skip binaryjoin, we're analyzing 2NSA now
        continue
    
    query = entry["params"]["query"]
    metrics = entry["plan"]["metrics"]
    row = {
        "method": method,   
        "query": int(query),
    }
    metrics = {"filter_time": 0, "projection_time": 0, "aggregate_time": 0, "parquet_time": 0, "coalesce_batches_time": 0}
    collect_timings(entry["plan"], metrics)
    # already convert all timings from ns to s
    metrics = {key: value / 1_000_000_000 for key, value in metrics.items()}
    row.update(metrics)
    projection.append(row)

yann_metrics = pd.DataFrame(projection)
yann_metrics["method"] = "SYA"
yann_metrics = yann_metrics.groupby(["query","method"]).aggregate("median")
yann_metrics.reset_index(inplace=True)

yann_metrics = pd.merge(
    df_agg[df_agg["method"] == "SYA"],
    yann_metrics,
    on=["query","method"]
)

# join_time = total_time - filter_time - projection_time - aggregate_time - parquet_time - coalesce_batches_time

yann_metrics["hashjoin_time"] = yann_metrics["execution_time"] - yann_metrics["aggregate_time"] - yann_metrics["filter_time"] - yann_metrics["projection_time"] - yann_metrics["parquet_time"] - yann_metrics["coalesce_batches_time"]
yann_metrics

,query,method,total_time,execution_time,filter_time,projection_time,aggregate_time,parquet_time,coalesce_batches_time,hashjoin_time
0,1,SYA,NaN,0.001255,0.000047,0.000000,0.000003,0.000272,0.000005,0.000927
1,2,SYA,NaN,0.020380,0.000882,0.000003,0.000227,0.001871,0.000049,0.017348
2,3,SYA,NaN,0.008881,0.001301,0.000002,0.000056,0.003033,0.000080,0.004409
3,4,SYA,NaN,0.010474,0.000808,0.000001,0.000011,0.004388,0.000061,0.005204
4,5,SYA,NaN,0.017128,0.001615,0.000001,0.000314,0.001891,0.000065,0.013241
...,...,...,...,...,...,...,...,...,...,...
138,142,SYA,NaN,2.138454,0.003491,0.000036,0.016375,0.008861,0.000200,2.109492
139,143,SYA,NaN,0.633652,0.003013,0.000008,0.011766,0.007291,0.000236,0.611339
140,144,SYA,NaN,0.089205,0.000884,0.000003,0.001272,0.003862,0.000055,0.083129
141,145,SYA,NaN,0.015764,0.002344,0.000007,0.000020,0.004597,0.000139,0.008657


In [8]:
yann_metrics.drop(columns=["filter_time","projection_time","aggregate_time","parquet_time","coalesce_batches_time"], inplace=True)
yann_metrics.to_csv('timings_agg_revision.csv', index=False, header=False, mode='a')
yann_metrics

,query,method,total_time,execution_time,hashjoin_time
0,1,SYA,NaN,0.001255,0.000927
1,2,SYA,NaN,0.020380,0.017348
2,3,SYA,NaN,0.008881,0.004409
3,4,SYA,NaN,0.010474,0.005204
4,5,SYA,NaN,0.017128,0.013241
...,...,...,...,...,...
138,142,SYA,NaN,2.138454,2.109492
139,143,SYA,NaN,0.633652,0.611339
140,144,SYA,NaN,0.089205,0.083129
141,145,SYA,NaN,0.015764,0.008657


## DuckDB runtimes

In [9]:
def total_mark_join_time(html):
    pattern = r"<b>\s*HASH JOIN\s*\(([\d|\.]+)s\)\s*<\/b>\s*</p>\s*<p>\s*MARK"
    mark_times = [float(match.group(1)) for match in re.finditer(pattern, html)]
    return sum(mark_times)

def extract_times_from_timing_table(html_file_path: str):
    with open(html_file_path, 'r') as file:
        content = file.read()
    
    soup = BeautifulSoup(content, 'html.parser')
    
    rows = soup.find_all('tr')
    
    total_time = None
    execution_time = None
    seq_scan_time = None
    hashjoin_time = None
    aggregate_time = None
    projection_time = None
    filter_time = None
    
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 1:
            phase = cells[0].get_text(strip=True)
            time = cells[1].get_text(strip=True)
            
            if phase == "TOTAL TIME":
                total_time = float(time)
            elif phase == 'Execution Time':
                execution_time = float(time)
            elif phase == 'SEQ_SCAN':
                seq_scan_time = float(time)
            elif phase == "HASH_JOIN":
                hashjoin_time = float(time)
            elif phase == "UNGROUPED_AGGREGATE":
                aggregate_time = float(time)
            elif phase == "PROJECTION":
                projection_time = float(time)
            elif phase == "FILTER":
                filter_time = float(time)

    mark_join_time = total_mark_join_time(content)
    
    return {
        'total_time': total_time,
        'execution_time': execution_time,
        'seq_scan_time': seq_scan_time,
        'hashjoin_time': hashjoin_time,
        "markjoin_time": mark_join_time,
        'aggregate_time': aggregate_time,
        'projection_time': projection_time,
        'filter_time': filter_time,
    }

def build_table(duckdb_plans: str):
    """ 
    Build Pandas DataFrame with timings reported by DuckDB html query plans.
    `duckdb_plans` is the path to the folder containing the html files.
    """
    duckdb_df = []

    for query_folder in os.listdir(duckdb_plans):
        query_folder_path = os.path.join(duckdb_plans, query_folder)
        if not os.path.isdir(query_folder_path):
            continue
        
        for query_file in os.listdir(query_folder_path):
            if not query_file.endswith('.html'):
                continue
            
            query_file_path = os.path.join(query_folder_path, query_file)
            times = extract_times_from_timing_table(query_file_path)
            duckdb_df.append(
                {
                    "query": query_folder,
                    "run": os.path.splitext(query_file)[0],
                    "total_time": times['total_time'],
                    "execution_time": times['execution_time'],
                    "aggregate_time": times['aggregate_time'],
                    "hashjoin_time": times["hashjoin_time"],
                    "markjoin_time": times["markjoin_time"],
                    "projection_time": times['projection_time'],
                    "filter_time": times['filter_time'],
                    "seq_scan_time": times['seq_scan_time'],
                }
            )

    duckdb_df = pd.DataFrame(duckdb_df)
    return duckdb_df

duckdb_plans = "../../query_plans/stats_duckdb/2_original_with_aliases"
duckdb_df = build_table(duckdb_plans)
duckdb_df.drop(columns=["run"], inplace=True)
duckdb_df["method"] = "DuckDB-Bin"

# Time (sec) spent on SequentialScan
print(duckdb_df["seq_scan_time"].describe())

# Percentage of time spent on SequentialScan
print((duckdb_df["seq_scan_time"] / duckdb_df["execution_time"] * 100).describe())


# # subtract markjoin time from hashjoin time to get the actual time spent in computing inner hashjoins
duckdb_df["hashjoin_time(s)"] = duckdb_df["hashjoin_time"] - duckdb_df["markjoin_time"]
duckdb_df.drop(columns=["aggregate_time","hashjoin_time","markjoin_time","projection_time","filter_time","seq_scan_time"], inplace=True)
duckdb_df.rename(columns={"hashjoin_time(s)":"hashjoin_time"}, inplace=True)

duckdb_df = duckdb_df.groupby(["query","method"]).aggregate("median")
duckdb_df.reset_index(inplace=True)
duckdb_df.to_csv('timings_agg_revision.csv', index=False, header=False, mode='a')
duckdb_df

count    1430.000000
mean        0.002333
std         0.001164
min         0.000144
25%         0.001355
50%         0.002316
75%         0.003116
max         0.006117
Name: seq_scan_time, dtype: float64
count    1430.000000
mean        9.127669
std        12.534560
min         0.001432
25%         0.407421
50%         3.494834
75%        13.332454
max        60.135454
dtype: float64


,query,method,total_time,execution_time,hashjoin_time
0,1,DuckDB-Bin,0.000946,0.000632,0.000481
1,10,DuckDB-Bin,0.109596,0.088338,0.086737
2,100,DuckDB-Bin,0.013926,0.011060,0.009396
3,101,DuckDB-Bin,0.030493,0.023280,0.020071
4,102,DuckDB-Bin,0.110422,0.085856,0.083115
...,...,...,...,...,...
138,95,DuckDB-Bin,0.159192,0.124332,0.121216
139,96,DuckDB-Bin,0.242524,0.189920,0.186113
140,97,DuckDB-Bin,3.041594,2.773124,2.765452
141,98,DuckDB-Bin,0.867178,0.788835,0.776176


## Umbra runtimes

In [10]:
import numpy as np
def parse_umbra_timings(file: str, benchmark: str, method: str) -> pd.DataFrame:
    df = pd.read_csv(file)
    df["benchmark"] = df["name"].apply(lambda x: x.split(":")[0])
    df["query"] = df["name"].apply(lambda x: x.split(":")[1].split(".")[0])
    df["method"] = method
    df["n_runs"] = df["execution_times"].apply(lambda x: len(json.loads(x)))
    df = df[df["n_runs"] == 10] # drop queries that did not run 10 times 
    df = df[df["benchmark"] == benchmark]
    df = df[["query","method","compilation_time_median","execution_time_median"]]
    df["total_time"] = df["compilation_time_median"] + df["execution_time_median"]
    df["hashjoin_time"]=np.nan
    df.rename(columns={"execution_time_median":"execution_time"}, inplace=True)
    return df[["query","method","total_time","execution_time","hashjoin_time"]]

In [11]:
method = "Umbra-Default"
file = "../../umbra/results/benchmark_umbra_default.csv"
benchmark = "statsceb"

umbra = parse_umbra_timings(file, benchmark, method)
umbra.to_csv('timings_agg_revision.csv', index=False, header=False, mode='a')
umbra

,query,method,total_time,execution_time,hashjoin_time
113,1,Umbra-Default,0.021876,0.002226,NaN
114,2,Umbra-Default,0.069199,0.050874,NaN
115,3,Umbra-Default,0.029784,0.009855,NaN
116,4,Umbra-Default,0.030730,0.011896,NaN
117,5,Umbra-Default,0.053665,0.033748,NaN
...,...,...,...,...,...
254,142,Umbra-Default,0.689678,0.635552,NaN
255,143,Umbra-Default,0.448387,0.392253,NaN
256,144,Umbra-Default,0.124145,0.076947,NaN
257,145,Umbra-Default,0.053164,0.005951,NaN


In [12]:
method = "Umbra-L&E"
file = "../../umbra/results/benchmark_umbra_le.csv"
benchmark = "statsceb"

umbra = parse_umbra_timings(file, benchmark, method)
umbra.to_csv('timings_agg_revision.csv', index=False, header=False, mode='a')
umbra

,query,method,total_time,execution_time,hashjoin_time
110,1,Umbra-L&E,0.070066,0.001236,NaN
111,2,Umbra-L&E,0.026795,0.010660,NaN
112,3,Umbra-L&E,0.019424,0.003403,NaN
113,4,Umbra-L&E,0.019851,0.004043,NaN
114,5,Umbra-L&E,0.023311,0.007563,NaN
...,...,...,...,...,...
251,142,Umbra-L&E,0.322704,0.257315,NaN
252,143,Umbra-L&E,0.202667,0.135835,NaN
253,144,Umbra-L&E,0.079446,0.030113,NaN
254,145,Umbra-L&E,0.053346,0.002722,NaN


In [13]:
method = "Umbra-Interpreted"
file = "../../umbra/results/benchmark_umbra_interpreted.csv"
benchmark = "statsceb"

umbra = parse_umbra_timings(file, benchmark, method)
umbra.to_csv('timings_agg_revision.csv', index=False, header=False, mode='a')
umbra

,query,method,total_time,execution_time,hashjoin_time
113,1,Umbra-Interpreted,0.008100,0.007112,NaN
114,2,Umbra-Interpreted,0.278467,0.277613,NaN
115,3,Umbra-Interpreted,0.049828,0.048968,NaN
116,4,Umbra-Interpreted,0.053444,0.052762,NaN
117,5,Umbra-Interpreted,0.197553,0.196892,NaN
...,...,...,...,...,...
254,142,Umbra-Interpreted,7.003553,7.001110,NaN
255,143,Umbra-Interpreted,4.098620,4.096210,NaN
256,144,Umbra-Interpreted,0.686510,0.684595,NaN
257,145,Umbra-Interpreted,0.023818,0.021660,NaN


In [14]:
method = "Umbra-Chained"
file = "../../umbra/results/benchmark_umbra_chained.csv"
benchmark = "statsceb"

umbra = parse_umbra_timings(file, benchmark, method)
umbra.to_csv('timings_agg_revision.csv', index=False, header=False, mode='a')
umbra

,query,method,total_time,execution_time,hashjoin_time
113,1,Umbra-Chained,0.013171,0.001563,NaN
114,2,Umbra-Chained,0.045138,0.033521,NaN
115,3,Umbra-Chained,0.017701,0.006221,NaN
116,4,Umbra-Chained,0.019072,0.007356,NaN
117,5,Umbra-Chained,0.034574,0.023034,NaN
...,...,...,...,...,...
254,142,Umbra-Chained,0.540757,0.505082,NaN
255,143,Umbra-Chained,0.344494,0.308264,NaN
256,144,Umbra-Chained,0.081987,0.053334,NaN
257,145,Umbra-Chained,0.034207,0.003451,NaN
